In [125]:
# Importation des modules nécessaires
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import statsmodels.api as sm

In [126]:
# Charger les fichier Excel avec les donnes de rendements et les poid de chaque actif dans le portefeuille et l'ETF 
df_EAFE = pd.read_excel('/Users/thomasdeconinck/Desktop/Fonds BNI/Fonds BNI Selection de Gestionnaire/Time Series/EAFE/Returns EAFE PORT Vs MSCI Index.xlsx', skiprows=4)

df_EM = pd.read_excel('/Users/thomasdeconinck/Desktop/Fonds BNI/Fonds BNI Selection de Gestionnaire/Time Series/EM/Returns BNI Vs MSCI EM Index.xlsx', skiprows=4)

In [1]:
# Fonction pour calculer la tracking error sur une fenêtre glissante
def rolling_tracking_error(series, window):
    return series.rolling(window=window).std(ddof=1)

# Fonction pour calculer l'Active Share sur une fenêtre glissante 
def rolling_active_share(df, window):
    active_shares = []
    for i in range(len(df) - window + 1):
        window_df = df.iloc[i:i+window]
        window_df['Difference Weight'] = np.abs(window_df['Portfolio Weights'] - window_df['iShares MSCI EAFE ETF Weights'])
        active_share = window_df['Difference Weight'].sum() / 2
        active_shares.append(active_share)
    # Ajouter des NaN pour les premières valeurs où la fenêtre n'est pas complète
    active_shares = [np.nan] * (window - 1) + active_shares
    return active_shares

def calculate_grinblatt_titman(fund_weights_t, fund_weights_t_minus_1, fund_returns, benchmark_returns):
    """
    Grinblatt-Titman (GT) Measure Calculation.
    
    Parameters:
    - fund_weights_t: DataFrame of portfolio weights at time t (current month)
    - fund_weights_t_minus_1: DataFrame of portfolio weights at time t-1 (previous month)
    - fund_returns: Series or DataFrame of stock returns
    - benchmark_returns: Series or DataFrame of benchmark returns
    
    Returns:
    - GT measure for the given period
    """
    # Calculate the change in portfolio weights over the period (1 month)
    weight_diff = fund_weights_t - fund_weights_t_minus_1
    
    # Multiply the weight differences by the excess returns (stock returns - benchmark returns)
    gt_contributions = weight_diff * (fund_returns - benchmark_returns)
    
    # Sum the GT contributions across all stocks to get the overall GT measure
    gt_measure = gt_contributions.sum()
    
    return gt_measure.mean()

def calculate_cs(fund_weights, stock_returns, benchmark_returns, size, book_to_market, momentum):
    """Calculates the Characteristic Selectivity (CS) measure."""
    # Create characteristic-based portfolios
    # Assume `size`, `book_to_market`, `momentum` are characteristics for each stock in the fund
    characteristics = pd.DataFrame({
        'size': size,
        'book_to_market': book_to_market,
        'momentum': momentum
    })

    # Define benchmark portfolios based on characteristics
    benchmark_portfolio_returns = benchmark_returns

    # Calculate excess return: (fund return - benchmark portfolio return)
    excess_returns = stock_returns - benchmark_portfolio_returns

    # Multiply excess returns by portfolio weights to get characteristic selectivity
    cs_measure = (fund_weights * excess_returns).sum(axis=1)

    return cs_measure.mean()

In [128]:
# Calculer la différence des rendements entre le fonds EAFE et l'indice de référence MSCI EAFE
# Calculer la variance des différences des rendements 
# Calculer la Tracking Error en prenant la racine carrée de la variance des différences des rendements 
df_EAFE['Difference Returns'] = df_EAFE['Portfolio Return (X)'] - df_EAFE['iShares MSCI EAFE ETF']

df_EM['Difference Returns'] = df_EM['Portfolio Return (X)'] - df_EM['iShares MSCI Emerging Markets ETF']

# Calculer la tracking error mensuelle avec une fenêtre de 12 mois
df_EAFE['Tracking Error'] = rolling_tracking_error(df_EAFE['Difference Returns'], window=12)
df_EM['Tracking Error'] = rolling_tracking_error(df_EM['Difference Returns'], window=12)

df_EAFE, df_EM

(         Date  Portfolio Return (X)  iShares MSCI EAFE ETF  \
 0  2018-10-31             -7.205487              -6.415308   
 1  2018-11-30              0.703917              -0.068933   
 2  2018-12-31             -2.315502              -4.816444   
 3  2019-01-31              4.969051               6.528877   
 4  2019-02-28              4.229240               2.568819   
 ..        ...                   ...                    ...   
 68 2024-06-30              1.774070              -1.576941   
 69 2024-07-31              0.627829               2.927931   
 70 2024-08-31              4.178848               3.256003   
 71 2024-09-30              0.460350               0.958445   
 72 2024-10-04             -2.471318              -2.239113   
 
     Book-to-Price (Portfolio) (Y)  Market Cap (Portfolio) (Y)  \
 0                        0.213883                6.337825e+10   
 1                        0.213158                6.477073e+10   
 2                        0.239070          

In [76]:
# Calculer l'Active Share mensuelle avec une fenêtre de 1 mois pour le fonds EAFE et EM
df_EAFE['Active Share'] = rolling_active_share(df_EAFE, window=12)
df_EM['Active Share'] = rolling_active_share(df_EM, window=12)

KeyError: 'Portfolio Weights'

In [129]:
def plot_management_styles(df, seuil_erreur_de_suivi=0.5, seuil_part_active=0.5):
    # Catégorisation des données basées sur les seuils fournis
    def categoriser(row):
        if row['Tracking Error'] <= seuil_erreur_de_suivi and row['Active Share'] <= seuil_part_active:
            return 'Indexation Pure'
        elif row['Tracking Error'] <= seuil_erreur_de_suivi and row['Active Share'] > seuil_part_active:
            return 'Sélection Diversifiée d\'Actions'
        elif row['Tracking Error'] > seuil_erreur_de_suivi and row['Active Share'] <= seuil_part_active:
            return 'Paris sur Facteurs'
        elif row['Tracking Error'] > seuil_erreur_de_suivi and row['Active Share'] > seuil_part_active:
            return 'Sélection Concentrée d\'Actions'
        else:
            return 'Indexation Déguisée'

    df['Catégorie'] = df.apply(categoriser, axis=1)

    # Tracé des données
    fig, ax = plt.subplots(figsize=(10, 6))
    couleurs = {
        'Indexation Pure': 'gray',
        'Indexation Déguisée': 'green',
        'Paris sur Facteurs': 'red',
        'Sélection Diversifiée d\'Actions': 'purple',
        'Sélection Concentrée d\'Actions': 'orange'
    }

    groupe = df.groupby('Catégorie')
    for cle, groupe in groupe:
        groupe.plot(ax=ax, kind='scatter', x='Tracking Error', y='Active Share', label=cle, color=couleurs[cle], alpha=0.6)

    # Ajout des étiquettes pour bas et haut
    plt.axhline(seuil_part_active, color='black', linestyle='--')
    plt.axvline(seuil_erreur_de_suivi, color='black', linestyle='--')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xlabel('Erreur de Suivi')
    ax.set_ylabel('Part Active')
    ax.set_title('Types de Styles de Gestion Active par Part Active et Erreur de Suivi')
    ax.grid(True)
    plt.show()

In [88]:
# Tracer les styles de gestion active
# Supposons que df_EAFE et df_EM possèdent aussi la colonne 'Active Share'
plot_management_styles(df_EAFE)
plot_management_styles(df_EM)